# SIRD: A Pandemic Model with Social Distancing
**Prof. Tony Saad (<a>www.tsaad.net</a>) <br/>Department of Chemical Engineering <br/>University of Utah**
<hr/>

In [1]:
#HIDDEN
from routines import plot_sird_model
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

continuousUpdate = False

beta = widgets.BoundedFloatText(value=0.5,min=0,max=1,step=0.01,description='Infection rate:',continuous_update=continuousUpdate)
style = {'description_width': 'initial'}
days = widgets.BoundedFloatText(value=14,min=1,max=20,step=1, description='Recovery (days):' ,style=style,continuous_update=continuousUpdate)
δ1 = widgets.BoundedFloatText(value=0.1,min=0,max=10.0,step=0.1,description='D to S ($\delta_1$):',readout_format='.3f',style=style,continuous_update=continuousUpdate)
δ2 = widgets.BoundedFloatText(value=0.15,min=0,max=10.0,step=0.1,description='S to D ($\delta_2$):',readout_format='.3f',style=style,continuous_update=continuousUpdate)
months = widgets.BoundedFloatText(value=12,min=1,max=60,step=1, description='Simulation (months):',style=style,continuous_update=continuousUpdate)
vaccinateAfter = widgets.BoundedFloatText(value=8,min=1,description='Vaccine found (months):',max=64,style=style,continuous_update=continuousUpdate)

minI = widgets.BoundedFloatText(value=1.0,min=0.0,max=50.0,step=0.01,description='Min Infected (%):',readout_format='.3f',style=style,continuous_update=continuousUpdate, disabled=True)
maxI = widgets.BoundedFloatText(value=10.0,min=0,max=50.0,step=0.01,description='Max Infected (%):',readout_format='.3f',style=style,continuous_update=continuousUpdate, disabled=True)

widgets.link((maxI, 'value'), (minI, 'max'))
def on_value_change(change):
    minI.max = maxI.value*0.99
maxI.observe(on_value_change, names='value')

distanceModel = widgets.Dropdown(options=[('Constant'), ('Reactive')],description='Social Distancing Model:',style=style)
def show_max_min(change):
    if distanceModel.value == 'Constant':
        maxI.disabled=True
        minI.disabled=True
    else:
        maxI.disabled=False
        minI.disabled=False
distanceModel.observe(show_max_min, names='value')

ui1 = widgets.HBox([beta,months])
ui2 = widgets.HBox([days,vaccinateAfter])
ui3 = widgets.HBox([δ1,δ2])
ui4 = widgets.HBox([minI,maxI])
ui5 = widgets.HBox([distanceModel])

out = widgets.interactive_output(plot_sird_model, {'infection_rate': beta, 'incubation_period': days, 'D_to_S': δ1, 'S_to_D': δ2, 'tend_months':months, 'vaccinateAfter': vaccinateAfter,'minIpercent':minI, 'maxIpercent':maxI,'distanceModel':distanceModel})

display(ui1,ui2,ui3,ui4, ui5, out)

Output()

The SIRD pandemic model is a variation of the SIR model with the addition of social distancing. A compartmental diagram is shown below:

<img src="sird-diagram.png" alt="SIRD Diagram" width="500"/>

The governing equations are:
\begin{equation}
\frac{\text{d}S}{\text{d}t} = -\beta SI - \delta_1 S + \delta_2 D
\end{equation}
\begin{equation}
\frac{\text{d}I}{\text{d}t} = \beta SI - \gamma I
\end{equation}
\begin{equation}
\frac{\text{d}R}{\text{d}t} =\gamma I
\end{equation}
\begin{equation}
\frac{\text{d}D}{\text{d}t} =\delta_1 S - \delta_2 D 
\end{equation}
where:
* $\beta$ is the average infection rate
* $\gamma$ is the fraction of people recovering, and is equal to $1/d$ where $d$ is the average number of days for recovery
* $\delta_1$ is the fraction at which people move from being socially distanced back to the general population
* $\delta_2$ is the average fraction at which people become socially distant

The Model implemented on this page triggers social distancing when the # of infected reaches a certain percentage of the population (designated as Max Infected in the GUI) and relaxes social distancing when it reaches a minimum # of infected (designated as Min Infected in the GUI).